Linguistic Research for "datast_tr.csv"

In [1]:
import pandas as pd


# Loading dataset
data_path = 'news.csv'
df = pd.read_csv(data_path, error_bad_lines=False)

# Calculating word numbers for each body
df["character_count"] = df["Body"].apply(lambda x: len(str(x)))
df["word_count"] = df["Body"].apply(lambda x: len(str(x).split()))
linguistic_data = df[["Body", "Label", "character_count", "word_count"]]


<ipython-input-1-367accbb58fa>:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(data_path, error_bad_lines=False)


In [2]:
df["letter_count"] = df["Body"].apply(lambda x: len([c for c in x if c.isalpha()]))

In [3]:
df["total_number_of_upper_characters"] = df["Body"].apply(lambda x: sum(1 for c in x if c.isupper()))

In [4]:
df["total_number_of_lower_characters"] = df["Body"].apply(lambda x: sum(1 for c in x if c.islower()))

In [5]:
df.head()

,Body,Label,character_count,word_count,letter_count,total_number_of_upper_characters,total_number_of_lower_characters
0,Karadeniz için tiyatro festivali zamanı! Devl...,gerçek,3691,476,2948,245,2703
1,Sözde Torbacının Haklı Gururu Dün gece 02 sula...,yalan,875,115,726,37,689
2,Buse'yi aramak için 400 bin liralık cihazını a...,gerçek,360,47,301,12,289
3,West Ham 3-0 Southampton (MAÇ ÖZET) Premier L...,gerçek,214,34,163,15,148
4,AK Partili Yavuz duyurdu: YSK'ya ek dilekçe ve...,gerçek,1105,143,925,53,872


In [6]:
import string

df["number_of_special_character"] = df["Body"].apply(lambda x: sum(1 for c in x if c in string.punctuation))

In [7]:
df["short_words"] = df["Body"].apply(lambda x: len([w for w in x.split() if len(w)<=4]))

In [8]:
df["long_words"] = df["Body"].apply(lambda x: len([w for w in x.split() if len(w)>4]))

In [9]:
df["number_of_different_words"] = df["Body"].apply(lambda x: len(set(x.split())))

In [10]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
#nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [11]:
import locale
locale.getdefaultlocale()

('en_US', 'UTF-8')

In [12]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [13]:
!pip install stanfordnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 4.2 MB/s eta 0:00:00


In [14]:
import stanfordnlp
stanfordnlp.download('tr')

Using the default treebank "tr_imst" for language "tr".
Would you like to download the models for: tr_imst now? (Y/n)
Y

Default download directory: /root/stanfordnlp_resources
Hit enter to continue or type an alternate directory.


Download location: /root/stanfordnlp_resources/tr_imst_models.zip


100%|██████████| 223M/223M [00:37<00:00, 5.91MB/s]



Download complete.  Models saved to: /root/stanfordnlp_resources/tr_imst_models.zip
Extracting models file for: tr_imst
Cleaning up...Done.


In [15]:
nlp = stanfordnlp.Pipeline(processors='tokenize,pos', lang='tr')

Use device: gpu
---
Loading: tokenize
With settings: 
{'model_path': '/root/stanfordnlp_resources/tr_imst_models/tr_imst_tokenizer.pt', 'lang': 'tr', 'shorthand': 'tr_imst', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/root/stanfordnlp_resources/tr_imst_models/tr_imst_tagger.pt', 'pretrain_path': '/root/stanfordnlp_resources/tr_imst_models/tr_imst.pretrain.pt', 'lang': 'tr', 'shorthand': 'tr_imst', 'mode': 'predict'}
Done loading processors!
---


In [16]:
unique_word_types_count = []
for article in df['Body']:
    word_types = set()
    doc = nlp(article)
    for sentence in doc.sentences:
        for word in sentence.words:
            if word.upos not in ['PUNCT', 'SYM', 'X']:
                word_types.add(word.upos)
    unique_word_types_count.append(len(word_types))

# Store the results in a new column
df['unique_word_types_count'] = unique_word_types_count

In [17]:
def get_verbs(text):
    doc = nlp(text)
    verbs = [word.text for sent in doc.sentences for word in sent.words if word.upos == 'VERB']
    return verbs

In [18]:
import stanfordnlp
def count_verbs(text):
    verbs = get_verbs(text)
    return len(verbs)

df['verb_count'] = df['Body'].apply(count_verbs)

In [19]:
def count_adjectives(text):
    doc = nlp(text)
    adjectives = [word.text for sentence in doc.sentences for word in sentence.words if word.upos == 'ADJ']
    return len(adjectives)

df['adjective_count'] = df['Body'].apply(count_adjectives)

In [20]:
def preprocess_text(text):
    # Tokenize 
    doc = nlp(text)
    tokens = [word.text for sent in doc.sentences for word in sent.words]
    
    # Remove stop words
    tokens = [token for token in tokens if token.lower() not in stop_words]
    
    # Split words into roots
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return lemmatized_tokens

df['determiners_count'] = df['Body'].apply(lambda x: len([word.upos for sent in nlp(x).sentences for word in sent.words if word.upos == 'DET']))


In [21]:
# Calculate the number of conjunctions
def count_conjunctions(text):
    doc = nlp(text)
    conjunctions = [word.text for sentence in doc.sentences for word in sentence.words if word.upos == 'CCONJ']
    return len(conjunctions)

df['conjunction_count'] = df['Body'].apply(count_conjunctions)

In [22]:
def count_nouns(text):
    doc = nlp(text)
    nouns = [word.text for sentence in doc.sentences for word in sentence.words if word.upos == 'NOUN']
    return len(nouns)

df['noun_count'] = df['Body'].apply(count_nouns)

In [23]:
df.head()

,Body,Label,character_count,word_count,letter_count,total_number_of_upper_characters,total_number_of_lower_characters,number_of_special_character,short_words,long_words,number_of_different_words,unique_word_types_count,verb_count,adjective_count,determiners_count,conjunction_count,noun_count
0,Karadeniz için tiyatro festivali zamanı! Devl...,gerçek,3691,476,2948,245,2703,71,125,351,315,10,39,29,4,38,234
1,Sözde Torbacının Haklı Gururu Dün gece 02 sula...,yalan,875,115,726,37,689,27,28,87,106,9,18,16,0,3,60
2,Buse'yi aramak için 400 bin liralık cihazını a...,gerçek,360,47,301,12,289,5,10,37,45,6,14,5,0,0,12
3,West Ham 3-0 Southampton (MAÇ ÖZET) Premier L...,gerçek,214,34,163,15,148,9,14,20,30,5,5,5,0,0,17
4,AK Partili Yavuz duyurdu: YSK'ya ek dilekçe ve...,gerçek,1105,143,925,53,872,33,40,103,108,11,32,17,2,9,51


In [24]:
def count_adverbs(text):
    doc = nlp(text)
    adverbs = [word.text for sentence in doc.sentences for word in sentence.words if word.upos == 'ADV']
    return len(adverbs)

df['adverb_count'] = df['Body'].apply(count_adverbs)

In [25]:
def count_prepositions(text):
    doc = nlp(text)
    prepositions = [word.text for sentence in doc.sentences for word in sentence.words if word.upos == 'ADP']
    return len(prepositions)

df['preposition_count'] = df['Body'].apply(count_prepositions)    

In [26]:
def count_pronouns(text):
    doc = nlp(text)
    pronouns = [word.text for sentence in doc.sentences for word in sentence.words if word.upos == 'PRON']
    return len(pronouns)
df['pronoun_count'] = df['Body'].apply(count_pronouns)    

In [27]:
df.head()#Check

,Body,Label,character_count,word_count,letter_count,total_number_of_upper_characters,total_number_of_lower_characters,number_of_special_character,short_words,long_words,number_of_different_words,unique_word_types_count,verb_count,adjective_count,determiners_count,conjunction_count,noun_count,adverb_count,preposition_count,pronoun_count
0,Karadeniz için tiyatro festivali zamanı! Devl...,gerçek,3691,476,2948,245,2703,71,125,351,315,10,39,29,4,38,234,5,4,0
1,Sözde Torbacının Haklı Gururu Dün gece 02 sula...,yalan,875,115,726,37,689,27,28,87,106,9,18,16,0,3,60,1,2,3
2,Buse'yi aramak için 400 bin liralık cihazını a...,gerçek,360,47,301,12,289,5,10,37,45,6,14,5,0,0,12,0,2,0
3,West Ham 3-0 Southampton (MAÇ ÖZET) Premier L...,gerçek,214,34,163,15,148,9,14,20,30,5,5,5,0,0,17,0,0,0
4,AK Partili Yavuz duyurdu: YSK'ya ek dilekçe ve...,gerçek,1105,143,925,53,872,33,40,103,108,11,32,17,2,9,51,2,7,6


In [28]:
import re
def count_numbers(text):
    doc = nlp(text)
    numbers = re.findall(r'\d+', text)
    return len(numbers)
df['number_count'] = df['Body'].apply(count_numbers)

Sentiment analysis of each news

In [29]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.4 MB/s eta 0:00:00


In [30]:
import transformers

In [31]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
model = AutoModelForSequenceClassification.from_pretrained("savasy/bert-base-turkish-sentiment-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-sentiment-cased", max_length=1024)

In [32]:
sa= pipeline("sentiment-analysis", tokenizer=tokenizer, model=model)

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [33]:
df['sentiment'] = df['Body'].apply(lambda x: sa(x[:512])[0]['label'])

In [34]:
linguistic_data = df[["Body","Label", "character_count", "word_count", "letter_count",
                      "total_number_of_upper_characters", "total_number_of_lower_characters",
                      "number_of_special_character", "short_words", "long_words", "number_of_different_words","unique_word_types_count"
                      ,"verb_count","adjective_count","determiners_count","conjunction_count","noun_count",
                      "adverb_count","preposition_count","pronoun_count",
                      "number_count","sentiment"]]

In [35]:
df.head(30)

,Body,Label,character_count,word_count,letter_count,total_number_of_upper_characters,total_number_of_lower_characters,number_of_special_character,short_words,long_words,...,verb_count,adjective_count,determiners_count,conjunction_count,noun_count,adverb_count,preposition_count,pronoun_count,number_count,sentiment
0,Karadeniz için tiyatro festivali zamanı! Devl...,gerçek,3691,476,2948,245,2703,71,125,351,...,39,29,4,38,234,5,4,0,53,positive
1,Sözde Torbacının Haklı Gururu Dün gece 02 sula...,yalan,875,115,726,37,689,27,28,87,...,18,16,0,3,60,1,2,3,3,negative
2,Buse'yi aramak için 400 bin liralık cihazını a...,gerçek,360,47,301,12,289,5,10,37,...,14,5,0,0,12,0,2,0,2,positive
3,West Ham 3-0 Southampton (MAÇ ÖZET) Premier L...,gerçek,214,34,163,15,148,9,14,20,...,5,5,0,0,17,0,0,0,6,negative
4,AK Partili Yavuz duyurdu: YSK'ya ek dilekçe ve...,gerçek,1105,143,925,53,872,33,40,103,...,32,17,2,9,51,2,7,6,0,positive
5,Onlar tüm Türkiye'nin annesi... Sinemamızın 10...,gerçek,1506,200,1244,33,1211,35,53,147,...,41,23,6,5,91,7,2,9,4,positive
6,Orucu bozan durumlar neler? Burun damlası oruc...,gerçek,1333,195,1075,24,1051,42,60,135,...,45,17,6,13,80,5,4,8,8,positive
7,"Ankara Emniyeti, Bayram Öncesinde Sahte Çocuk ...",yalan,2864,374,2386,66,2320,87,98,276,...,83,49,7,18,153,23,10,9,10,negative
8,Her Şeyin En İyisini Hakettiğini Düşünen Genç ...,yalan,5083,675,4279,81,4198,104,202,473,...,137,101,14,43,233,43,26,19,16,positive
9,Suriye sınırına komando ve askeri araç sevkiya...,gerçek,520,62,446,19,427,12,8,54,...,8,9,0,5,33,0,0,0,0,positive


In [36]:
print(linguistic_data.head())

                                                Body   Label  character_count  \
0  Karadeniz için tiyatro festivali zamanı!  Devl...  gerçek             3691   
1  Sözde Torbacının Haklı Gururu Dün gece 02 sula...   yalan              875   
2  Buse'yi aramak için 400 bin liralık cihazını a...  gerçek              360   
3  West Ham 3-0 Southampton (MAÇ ÖZET)  Premier L...  gerçek              214   
4  AK Partili Yavuz duyurdu: YSK'ya ek dilekçe ve...  gerçek             1105   

   word_count  letter_count  total_number_of_upper_characters  \
0         476          2948                               245   
1         115           726                                37   
2          47           301                                12   
3          34           163                                15   
4         143           925                                53   

   total_number_of_lower_characters  number_of_special_character  short_words  \
0                              2703      

Adding "NER" Models...


In [37]:
!pip install spacy
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-06-03 17:40:37.546666: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 94.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [38]:
import spacy
import pandas as pd

# "en_core_web_sm" 
nlp = spacy.load("en_core_web_sm")

# List for entities.
entities = []

# Apply NER for "Body" column.
for text in df["Body"]:
    doc = nlp(text)
    for ent in doc.ents:
        entities.append((ent.text, ent.label_))
        
# Dataframe.
entities_df = pd.DataFrame(entities, columns=["entity", "label"])

print(entities_df.groupby("label").count())

             entity
label              
CARDINAL      22620
DATE           3980
EVENT           274
FAC            1447
GPE           25047
LANGUAGE        116
LAW             179
LOC             849
MONEY           159
NORP           8868
ORDINAL         100
ORG           40210
PERCENT         184
PERSON       133250
PRODUCT        3232
QUANTITY       1393
TIME            225
WORK_OF_ART    1950


This output contains numbers according to the entity types determined by the NER model.


*   CARDINAL: numeric values

*   DATE: dates

*   EVENT: events

*   FAC: structures and facilities

*   GPE: countries, cities and regions

*   LANGUAGE: languages

*   LAW: legal terms

*   LOC: geolocations

*   MONEY: currencies

*   NORP: nationality and ethnic groups

*   ORDINAL: rank numbers

*   ORG: companies, organizations and institutions

*   PERCENT: percentage values

*   PERSON: persons

*   PRODUCT: products and services

*   QUANTITY: units of measure

*   TIME: hours, days, etc. time expressions

*   WORK_OF_ART: books, movies, etc. artworks



For example, the most detected entity type in this dataset was "PERSON" and there are 152532 "PERSON" entities in total.

In [39]:
import spacy
import pandas as pd

# Load English NLP model
nlp = spacy.load("en_core_web_sm")

# Define function to get entity counts from a text string
def get_entity_counts(row):
    doc = nlp(row["Body"])
    entity_counts = {label:0 for label in nlp.pipe_labels["ner"]}
    for ent in doc.ents:
        entity_counts[ent.label_] += 1
    return pd.Series(entity_counts)

# Read in the data
df = linguistic_data[[ "Body","Label", "character_count", "word_count", "letter_count",
                      "total_number_of_upper_characters", "total_number_of_lower_characters",
                      "number_of_special_character", "short_words", "long_words", "number_of_different_words","unique_word_types_count"
                      ,"verb_count","adjective_count","determiners_count","conjunction_count","noun_count",
                      "adverb_count","preposition_count","pronoun_count",
                      "number_count","sentiment"]]

# Apply the function to each row of the dataframe to get entity counts
entity_counts = df.apply(get_entity_counts, axis=1)

# Combine the entity counts with the original dataframe
new_data = pd.concat([df, entity_counts], axis=1)


In [40]:
print(new_data.head())

                                                Body   Label  character_count  \
0  Karadeniz için tiyatro festivali zamanı!  Devl...  gerçek             3691   
1  Sözde Torbacının Haklı Gururu Dün gece 02 sula...   yalan              875   
2  Buse'yi aramak için 400 bin liralık cihazını a...  gerçek              360   
3  West Ham 3-0 Southampton (MAÇ ÖZET)  Premier L...  gerçek              214   
4  AK Partili Yavuz duyurdu: YSK'ya ek dilekçe ve...  gerçek             1105   

   word_count  letter_count  total_number_of_upper_characters  \
0         476          2948                               245   
1         115           726                                37   
2          47           301                                12   
3          34           163                                15   
4         143           925                                53   

   total_number_of_lower_characters  number_of_special_character  short_words  \
0                              2703      

----------------


In [42]:
new_data.to_csv('new_linguistic_data.csv')